In [1]:
import torch
print(torch.cuda.is_available())
print(torch.cuda.get_device_name(0))


True
NVIDIA A100-SXM4-80GB


In [124]:
import torch

print("Is CUDA available?", torch.cuda.is_available())
print("CUDA device count:", torch.cuda.device_count())
print("Current device ID:", torch.cuda.current_device())
print("Current device name:", torch.cuda.get_device_name(torch.cuda.current_device()))


Is CUDA available? True
CUDA device count: 8
Current device ID: 0
Current device name: NVIDIA A100-SXM4-80GB


In [4]:
import pandas as pd
import json

#Observe final Dataset
df_modbus = pd.read_json("modbus_dataset.jsonl", lines=True)
df_modbus.head()

,query,response
0,060f000000060005fffdff00,060f00000003008502
1,01b7000000060005fffdff00,01b700000003008502
2,05420000000d0010ec88000306831883180000,054200000003009002
3,005f0000000d0010fffd000306ffffffffffff,005f00000003009002
4,05c40000000b00100026000204ffffffff,05c400000006001000260002


In [5]:
from huggingface_hub import hf_hub_download

model_path = hf_hub_download(
    repo_id="RWKV/rwkv-5-world-all-pth",
    filename="RWKV-5-World-0.4B-v2-20231113-ctx4096.pth"
)

print(f"Model downloaded to: {model_path}")


(…)WKV-5-World-0.4B-v2-20231113-ctx4096.pth:   0%|          | 0.00/924M [00:00<?, ?B/s]

Model downloaded to: /home/re2230/.cache/huggingface/hub/models--RWKV--rwkv-5-world-all-pth/snapshots/d48d1c54cf32cfdc13fca3c767998a0bf3eef8b6/RWKV-5-World-0.4B-v2-20231113-ctx4096.pth


In [6]:
model_path = "/home/re2230/.cache/huggingface/hub/models--RWKV--rwkv-5-world-all-pth/snapshots/d48d1c54cf32cfdc13fca3c767998a0bf3eef8b6/RWKV-5-World-0.4B-v2-20231113-ctx4096.pth"


In [8]:
import json
print(json.loads('{"mode":"bone","r":32,"load":""}'))


{'mode': 'bone', 'r': 32, 'load': ''}


In [36]:
cd /home/re2230/RWKV-PEFT/


/home/re2230/RWKV-PEFT


/home/re2230/rwkv-env/lib/python3.10/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [26]:
import wandb
wandb.login()


wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /home/re2230/.netrc
wandb: Currently logged in as: re2230 (re2230-new-york-university) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

In [43]:
cd /home/re2230


/home/re2230


/home/re2230/rwkv-env/lib/python3.10/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [58]:
import torch
print(torch.cuda.is_bf16_supported())

True


In [ ]:
import subprocess
import shlex
import os

# Activar seguimiento en Weights & Biases
os.environ["WANDB_PROJECT"] = "rwkv-finetune"

from datasets import load_dataset

dataset = load_dataset("json", data_files="modbus_dataset.jsonl")

model_path = "/home/re2230/.cache/huggingface/hub/models--RWKV--rwkv-5-world-all-pth/snapshots/d48d1c54cf32cfdc13fca3c767998a0bf3eef8b6/RWKV-5-World-0.4B-v2-20231113-ctx4096.pth"

command = f"""
python RWKV-PEFT/train.py \
--load_model {model_path} \
--data_file json \
--data_files /home/re2230/modbus_dataset.jsonl \
--data_type sft \
--sft_field query response \
--ctx_len 512 \
--epoch_steps 100 \
--epoch_count 5 \
--micro_bsz 1 \
--n_layer 24 \
--n_embd 1024 \
--vocab_size 65536 \
--lr_init 6e-5 \
--warmup_steps 10 \
--strategy deepspeed \
--accelerator cuda \
--precision bf16 \
--peft disha \
--disha_config '{{"mode":"bone","r":32,"load":""}}' \
--proj_dir out/rwkv-modbus-ft
"""


result = subprocess.run(shlex.split(command), capture_output=True, text=True)

print("📤 STDOUT:\n", result.stdout)
print("⚠️ STDERR:\n", result.stderr)

if result.returncode != 0:
    print(f"❌ Training failed with exit code {result.returncode}")
else:
    print("✅ Training started successfully")   


📤 STDOUT:
 [2025-05-27 10:58:57,088] [INFO] [real_accelerator.py:110:get_accelerator] Setting ds_accelerator to cuda (auto detect)
########## WKV OP           cuda               ##########
########## WKV OP           cuda               ##########
########## WKV OP           cuda               ##########

########## FUSED OP    False          ##########
########## FUSED OP    False          ##########
########## FUSED OP    False          ##########

RWKV_MY_TESTING x052
ninja: no work to do.
RWKV(
  (model): RWKV5(
    (emb): Embedding(65536, 1024)
    (blocks): ModuleList(
      (0): Block(
        (ln1): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
        (ln2): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
        (ln0): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
        (att): RWKV_TimeMix_RWKV5(
          (time_shift): ZeroPad2d((0, 0, 1, -1))
          (receptance): BoneLinear()
          (key): BoneLinear()
          (value): BoneLinear()
        

In [90]:
import sys
import os
sys.path.append("/home/re2230/RWKV-PEFT/rwkvt/dataset")  # change to your actual path


In [88]:
import os
# 🔧 Variables de entorno necesarias para RWKV-PEFT
os.environ["RWKV_HEAD_SIZE_A"] = "64"
os.environ["RWKV_MY_TESTING"] = "x052"

In [105]:
import os
import json
import torch
from transformers import AutoTokenizer
from rwkvt.peft.peft_loading import load_peft_model
from rwkvt.args_type import TrainingArgs
from rwkv.utils import PIPELINE

IGNORE_INDEX = -100
EOT_TOKEN = "\x17"
PROMPT = (
    "Below is an instruction that describes a task. "
    "Write a response that appropriately completes the request.\n\n"
    "### Instruction:\n{instruction}\n\n### Response:"
)

# ⚙️ Setup model args
args = TrainingArgs()
args.load_model = "/home/re2230/out/rwkv-modbus-ft/rwkv-4.pth"
args.strategy = "cuda bf16"
args.ctx_len = 512
args.peft = "disha"
args.n_layer = 24
args.n_embd = 1024
args.vocab_size = 65536
args.dim_att = 1024
args.dim_ffn = 3584
args.head_size_a = 64
args.head_size_divisor = 8
os.environ["RWKV_HEAD_SIZE_A"] = str(args.head_size_a)
args.disha_config = {"mode": "bone", "r": 32, "load": ""}
os.environ["RWKV_MY_TESTING"] = "x052"
os.environ["RWKV_HEAD_SIZE_A"] = "64"
os.environ["RWKV_TRAIN_TYPE"] = "sft"

# 🧪 Evaluation loop
args, model = load_peft_model(args)
model = model.to(dtype=torch.bfloat16, device="cuda").eval()

# model.eval().cuda()

# 🧠 Cargar tokenizer (misma ruta usada en entrenamiento)
tokenizer = AutoTokenizer.from_pretrained("RWKV/rwkv-5-world-3b", trust_remote_code=True)

# 📄 Evaluación con archivo jsonl
PROMPT = (
    "Below is an instruction that describes a task. "
    "Write a response that appropriately completes the request.\n\n"
    "### Instruction:\n{instruction}\n\n### Response:"
)

test_file = "/home/re2230/RWKV-PEFT/rwkvt/dataset/modbus_dataset_test.jsonl"
correct = 0
total = 0

with open(test_file, "r") as f:
    for line in f:
        item = json.loads(line)
        prompt = PROMPT.format_map({"instruction": item["query"]})
        expected = item["response"]

        # 🔁 Tokenizar
        input_ids = tokenizer(prompt, return_tensors="pt").input_ids.cuda()

        # 🔮 Inferencia
# 🔮 Inferencia
        with torch.no_grad():
            input_ids = tokenizer(prompt, return_tensors="pt")["input_ids"].to("cuda")  # Mantener como int64

            # El modelo automáticamente opera en bfloat16 si se cargó con strategy="cuda bf16"
            logits = model(input_ids)[0]
            pred_ids = torch.argmax(logits, dim=-1)
            output = tokenizer.decode(pred_ids[0], skip_special_tokens=True)

        # 🎯 Comparación exacta
        if output.strip() == expected.strip():
            correct += 1
        total += 1

print(f"✅ Accuracy (BCA): {correct / total:.4f} ({correct}/{total})")


########## Loading /home/re2230/out/rwkv-modbus-ft/rwkv-4.pth... ##########


RWKV(
  (model): RWKV5(
    (emb): Embedding(65536, 1024)
    (blocks): ModuleList(
      (0): Block(
        (ln1): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
        (ln2): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
        (ln0): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
        (att): RWKV_TimeMix_RWKV5(
          (time_shift): ZeroPad2d((0, 0, 1, -1))
          (receptance): BoneLinear()
          (key): BoneLinear()
          (value): BoneLinear()
          (output): BoneLinear()
          (gate): BoneLinear()
          (ln_x): GroupNorm(16, 1024, eps=1e-05, affine=True)
        )
        (ffn): RWKV_ChannelMix(
          (time_shift): ZeroPad2d((0, 0, 1, -1))
          (key): BoneLinear()
          (receptance): BoneLinear()
          (value): BoneLinear()
        )
      )
      (1-23): 23 x Block(
        (ln1): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
        (ln2): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
     

In [110]:
from pathlib import Path

test_file = "/home/re2230/RWKV-PEFT/rwkvt/dataset/modbus_dataset_test.jsonl"
correct = 0
total = 0

output_file = Path("modbus_predictions.txt")
with open(output_file, "w") as out_f:
    out_f.write("Prompt\tExpected\tPredicted\tMatch\n")

with open(test_file, "r") as f:
    for line in f:
        item = json.loads(line)
        prompt = PROMPT.format_map({"instruction": item["query"]})
        expected = item["response"]

        # 🔁 Tokenizar
        input_ids = tokenizer(prompt, return_tensors="pt").input_ids.cuda()

        # 🔮 Inferencia

        with torch.no_grad():
            input_ids = tokenizer(prompt, return_tensors="pt")["input_ids"].to("cuda")  # Mantener como int64

            # El modelo automáticamente opera en bfloat16 si se cargó con strategy="cuda bf16"
            logits = model(input_ids)[0]
            pred_ids = torch.argmax(logits, dim=-1)
            output = tokenizer.decode(pred_ids[0], skip_special_tokens=True)

            match = output.strip() == expected.strip()
            if match:
                correct += 1
            total += 1

            # Save to file
            with open(output_file, "a") as out_f:
                out_f.write(f"{item['query']}\t{expected.strip()}\t{output.strip()}\t{match}\n")


print(f"✅ Accuracy (BCA): {correct / total:.4f} ({correct}/{total})")




✅ Accuracy (BCA): 0.0000 (0/160)


In [113]:
from pathlib import Path
import json
import torch

test_file = "/home/re2230/RWKV-PEFT/rwkvt/dataset/modbus_dataset_test.jsonl"
output_file = Path("modbus_predictions.txt")

correct = 0
total = 0

# Define column widths
col_widths = {
    "prompt": 40,
    "expected": 40,
    "predicted": 40,
    "match": 7,
}

# Write header
with open(output_file, "w") as out_f:
    header = f"{'Prompt'.ljust(col_widths['prompt'])} | {'Expected'.ljust(col_widths['expected'])} | {'Predicted'.ljust(col_widths['predicted'])} | {'Match'}\n"
    out_f.write(header)
    out_f.write("-" * len(header) + "\n")

# Inference loop
with open(test_file, "r") as f:
    for line in f:
        item = json.loads(line)
        prompt = PROMPT.format_map({"instruction": item["query"]})
        expected = item["response"]

        input_ids = tokenizer(prompt, return_tensors="pt")["input_ids"].to("cuda")

        with torch.no_grad():
            logits = model(input_ids)[0]
            pred_ids = torch.argmax(logits, dim=-1)
            output = tokenizer.decode(pred_ids[0], skip_special_tokens=True)
            print(output.strip())

        match = output.strip() == expected.strip()
        if match:
            correct += 1
        total += 1

        # Truncate long values for readability
        q = item['query'][:col_widths["prompt"] - 3] + "..." if len(item['query']) > col_widths["prompt"] else item['query']
        e = expected.strip()[:col_widths["expected"] - 3] + "..." if len(expected.strip()) > col_widths["expected"] else expected.strip()
        p = output.strip()[:col_widths["predicted"] - 3] + "..." if len(output.strip()) > col_widths["predicted"] else output.strip()

        with open(output_file, "a") as out_f:
            row = f"{q.ljust(col_widths['prompt'])} | {e.ljust(col_widths['expected'])} | {p.ljust(col_widths['predicted'])} | {str(match)}\n"
            out_f.write(row)

print(f"✅ Accuracy (BCA): {correct / total:.4f} ({correct}/{total})")




























































































































































✅ Accuracy (BCA): 0.0000 (0/160)


In [122]:
import os
import json
import torch
from transformers import AutoTokenizer
from rwkvt.peft.peft_loading import load_peft_model
from rwkvt.args_type import TrainingArgs
from pathlib import Path

# === Setup fine-tuned model ===
args = TrainingArgs()
args.load_model = "/home/re2230/out/rwkv-modbus-ft/rwkv-4.pth"
args.strategy = "cuda bf16"
args.ctx_len = 512
args.peft = "disha"
args.n_layer = 24
args.n_embd = 1024
args.vocab_size = 65536
args.dim_att = 1024
args.dim_ffn = 3584
args.head_size_a = 64
args.head_size_divisor = 8
args.disha_config = {"mode": "bone", "r": 32, "load": ""}

# Required env vars
os.environ["RWKV_MY_TESTING"] = "x052"
os.environ["RWKV_HEAD_SIZE_A"] = str(args.head_size_a)
os.environ["RWKV_TRAIN_TYPE"] = "sft"

# Load model
args, model = load_peft_model(args)
model = model.eval().cuda().to(dtype=torch.bfloat16)


# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained("RWKV/rwkv-5-world-3b", trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token

# Prompt template
PROMPT = (
    "Below is an instruction that describes a task. "
    "Write a response that appropriately completes the request.\n\n"
    "### Instruction:\n{instruction}\n\n### Response:"
)

# Evaluation
test_file = "/home/re2230/RWKV-PEFT/rwkvt/dataset/modbus_dataset_test.jsonl"
output_file = Path("modbus_predictions_2.txt")
correct, total = 0, 0

with open(output_file, "w") as out_f:
    out_f.write("Prompt\tExpected\tPredicted\tMatch\n")

    with open(test_file, "r") as f:
        for line in f:
            item = json.loads(line)
            prompt_text = PROMPT.format_map({"instruction": item["query"]})
            expected = item["response"]

            # Encode and initialize sequence
            input_ids = tokenizer(prompt_text, return_tensors="pt")["input_ids"].to(device="cuda")  # 🟢 Keep int64!
            generated = input_ids.clone()

            # Greedy decoding
            for _ in range(len(expected) + 4):
                with torch.no_grad(), torch.autocast(device_type="cuda", dtype=torch.bfloat16):
                    logits = model(generated)[0]
                    next_token_logits = logits[-1, :]
                    next_token_id = torch.argmax(next_token_logits, dim=-1).unsqueeze(0).unsqueeze(0)
                    generated = torch.cat([generated, next_token_id], dim=1)

                    if next_token_id.item() == tokenizer.eos_token_id:
                        break

            # Decode predicted
            predicted_text = tokenizer.decode(generated[0][input_ids.shape[1]:], skip_special_tokens=True).strip()
            match = predicted_text == expected.strip()

            if match:
                correct += 1
            total += 1

            # Write result
            out_f.write(f"{item['query']}\t{expected.strip()}\t{predicted_text}\t{match}\n")

print(f"✅ Accuracy (BCA): {correct / total:.4f} ({correct}/{total})")


########## Loading /home/re2230/out/rwkv-modbus-ft/rwkv-4.pth... ##########


RWKV(
  (model): RWKV5(
    (emb): Embedding(65536, 1024)
    (blocks): ModuleList(
      (0): Block(
        (ln1): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
        (ln2): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
        (ln0): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
        (att): RWKV_TimeMix_RWKV5(
          (time_shift): ZeroPad2d((0, 0, 1, -1))
          (receptance): BoneLinear()
          (key): BoneLinear()
          (value): BoneLinear()
          (output): BoneLinear()
          (gate): BoneLinear()
          (ln_x): GroupNorm(16, 1024, eps=1e-05, affine=True)
        )
        (ffn): RWKV_ChannelMix(
          (time_shift): ZeroPad2d((0, 0, 1, -1))
          (key): BoneLinear()
          (receptance): BoneLinear()
          (value): BoneLinear()
        )
      )
      (1-23): 23 x Block(
        (ln1): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
        (ln2): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
     

In [123]:
import os
import json
import torch
from pathlib import Path
from transformers import AutoTokenizer
from rwkvt.peft.peft_loading import load_peft_model
from rwkvt.args_type import TrainingArgs

# === Model Setup ===
args = TrainingArgs()
args.load_model = "/home/re2230/out/rwkv-modbus-ft/rwkv-4.pth"
args.strategy = "cuda bf16"
args.ctx_len = 512
args.peft = "disha"
args.n_layer = 24
args.n_embd = 1024
args.vocab_size = 65536
args.dim_att = 1024
args.dim_ffn = 3584
args.head_size_a = 64
args.head_size_divisor = 8
args.disha_config = {"mode": "bone", "r": 32, "load": ""}

# === Environment Variables ===
os.environ["RWKV_MY_TESTING"] = "x052"
os.environ["RWKV_HEAD_SIZE_A"] = str(args.head_size_a)
os.environ["RWKV_TRAIN_TYPE"] = "sft"

# === Load Model ===
args, model = load_peft_model(args)
model = model.eval().to(dtype=torch.bfloat16, device="cuda")

# === Load Tokenizer ===
tokenizer = AutoTokenizer.from_pretrained("RWKV/rwkv-5-world-3b", trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token  # Optional

# === Prompt Template ===
PROMPT = (
    "Below is an instruction that describes a task. "
    "Write a response that appropriately completes the request.\n\n"
    "### Instruction:\n{instruction}\n\n### Response:"
)

# === Evaluation ===
test_file = "/home/re2230/RWKV-PEFT/rwkvt/dataset/modbus_dataset_test.jsonl"
output_file = Path("modbus_predictions_log.tsv")
correct = 0
total = 0

with open(test_file, "r") as f, open(output_file, "w") as out_f:
    out_f.write("Prompt\tExpected\tPredicted\tMatch\n")
    out_f.write("=" * 150 + "\n")

    for line in f:
        item = json.loads(line)
        instruction = item["query"]
        expected = item["response"].strip()
        prompt_text = PROMPT.format(instruction=instruction)

        # Tokenize once
        input_ids = tokenizer(prompt_text, return_tensors="pt")["input_ids"].to("cuda")

        with torch.no_grad():
            logits = model(input_ids)[0]  # RWKV returns [seq_len, vocab_size]
            pred_ids = torch.argmax(logits, dim=-1)
            predicted_text = tokenizer.decode(pred_ids[0], skip_special_tokens=True).strip()

        match = predicted_text == expected
        correct += int(match)
        total += 1

        # Log tabular result
        out_f.write(f"{instruction}\t{expected}\t{predicted_text}\t{match}\n")

# === Final Score ===
print(f"✅ Accuracy (Exact Match): {correct / total:.4f} ({correct}/{total})")
print(f"📄 Predictions saved to: {output_file.resolve()}")


########## Loading /home/re2230/out/rwkv-modbus-ft/rwkv-4.pth... ##########


RWKV(
  (model): RWKV5(
    (emb): Embedding(65536, 1024)
    (blocks): ModuleList(
      (0): Block(
        (ln1): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
        (ln2): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
        (ln0): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
        (att): RWKV_TimeMix_RWKV5(
          (time_shift): ZeroPad2d((0, 0, 1, -1))
          (receptance): BoneLinear()
          (key): BoneLinear()
          (value): BoneLinear()
          (output): BoneLinear()
          (gate): BoneLinear()
          (ln_x): GroupNorm(16, 1024, eps=1e-05, affine=True)
        )
        (ffn): RWKV_ChannelMix(
          (time_shift): ZeroPad2d((0, 0, 1, -1))
          (key): BoneLinear()
          (receptance): BoneLinear()
          (value): BoneLinear()
        )
      )
      (1-23): 23 x Block(
        (ln1): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
        (ln2): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
     